# General DA fittings and Fokker-Planck comparisons (Hénon map)

## Setup scripts if we are under SWAN

In [ ]:
# Working in the right path
%cd /eos/project/d/da-and-diffusion-studies/DA_Studies/Simulations/Models/da_sixtrack

In [ ]:
# Install the libraries
import sys
!{sys.executable} -m pip install --user tqdm pynverse sixtrackwrap crank-nicolson-numba henon-map
!export PYTHONPATH=$CERNBOX_HOME/.local/lib/python3.7/site-packages:$PYTHONPATH

## Imports

In [1]:
%matplotlib widget

In [ ]:
# Base libraries
import math
import numpy as np
import scipy.integrate as integrate
from scipy.special import erf
import pickle
import itertools
from scipy.optimize import curve_fit

from numba import njit, prange

# Personal libraries
import sixtrackwrap as sx
import crank_nicolson_numba.nekhoroshev as nk

from tqdm.notebook import tqdm
import time
import matplotlib.pyplot as plt
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.ticker as ticker
from math import gcd

import pandas as pd

from scipy.special import lambertw
from scipy.interpolate import interp1d

import warnings
import os

# General setup
## Prepare inverse functions for obtaining DA from loss values

In [ ]:
from pynverse import inversefunc

# Uniform 4D distribution

@njit
def uniform_loss(DA, DA0):
    return (DA ** 4 / DA0 ** 4)

@njit
def DA_from_unifom_loss(loss, DA0):
    return np.power((loss * DA0 ** 4), 1/4)

# Symmetric 4D gaussian

@njit
def symmetric_gaussian_loss(DA, sigma, DA0):
    baseline = - np.exp(- 0.5 * (DA0 / sigma) ** 2) * (DA0 ** 2 + 2 * sigma ** 2) + 2 * sigma ** 2
    return (- np.exp(- 0.5 * (DA / sigma) ** 2) * (DA ** 2 + 2 * sigma ** 2) + 2 * sigma ** 2) / baseline


def DA_from_symmetric_gaussian_loss(loss, sigma, DA0):
    func = inversefunc(
        lambda x: symmetric_gaussian_loss(x, sigma, DA0),
        domain=[0.0, DA0]
    )
    return func(loss)
    

## Define Fitting Models!

### $\chi^2$ function

In [ ]:
def chi_2(original, estimate):
    return np.sum(np.power(estimate - original, 2) / original)

### Model 2

$$D(N) = \rho_\ast \left(\frac{\kappa}{2e}\right)^\kappa \frac{1}{\ln^\kappa\frac{N}{N_0}}$$


In [ ]:
@njit
def model_2(x, rho, n0, k):
    return rho * np.power(k / (2 * np.exp(1)), k) / (np.power(np.log(x / n0), k))


def explore_k_model_2(turns, da, k_min, k_max, n_samples):
    ks = np.linspace(k_min, k_max, n_samples)
    pars = []
    errs = []
    co_pars = []
    for k in tqdm(ks):
        par, co_par = curve_fit(
            lambda x, a, b : model_2(x, a, b, k),
            turns,
            da,
            bounds=([0, 0.00001],[np.inf, turns[-1]-0.0001])
        )
        pars.append(par)
        co_pars.append(co_par)
        errs.append(chi_2(da, model_2(turn_sampling, par[0], par[1], k)))
    return np.asarray(pars), np.asarray(errs), np.asarray(co_pars)

### Model 4

$$D(N)=\rho_{*} \times \frac{1}{\left[-2 \mathrm{e} \lambda \mathcal{W}_{-1}\left(-\frac{1}{2 \mathrm{e} \lambda}\left(\frac{\rho_{\ast}}{6}\right)^{1 / \kappa}\left(\frac{8}{7} N\right)^{-1 /(\lambda \kappa)}\right)\right]^{\kappa}}
$$

(with $\lambda=0.5$)

In [ ]:
def model_4(x, rho, k):
    lamb = 0.5
    return np.real(
        rho
        / np.power(-2 * np.exp(1) * lamb * lambertw(- (1 / (2 * np.exp(1) * lamb)) * np.power(rho / 6, 1 / k) * np.power((8/7) * x, -1 / (lamb * k)), -1), k)
    )

def explore_k_model_4(turns, da, k_min, k_max, n_samples):
    ks = np.linspace(k_min, k_max, n_samples)
    pars = []
    errs = []
    co_pars = []
    for k in tqdm(ks):
        par, co_par = curve_fit(
            lambda x, a : model_4(x, a, k),
            turns,
            da,
            bounds=([0.1],[np.inf])
        )
        pars.append(par)
        co_pars.append(co_par)
        errs.append(chi_2(da, model_4(turn_sampling, par[0], k)))
    return np.asarray(pars), np.asarray(errs), np.asarray(co_pars)

### A more free Model 4

$$D(N)=\rho_{*} \times \frac{1}{\left[-2 \mathrm{e} \lambda \mathcal{W}_{-1}\left(-\frac{1}{\lambda\kappa}\left(\frac{N}{N_0}\right)^{-\frac{1}{\lambda\kappa}}\right)\right]^{\kappa}}
$$

(with $\lambda=0.5$)

In [7]:
def model_4_free(x, rho, n0, k):
    lamb = 0.5
    temp = (
        rho / (np.power(- 2 * lamb * np.exp(1) * lambertw(- (1 / (lamb * k)) * np.power(x / n0, - 1 / (lamb * k)), -1), k))
    )
    
    return np.real(temp) # TODO: FIX IT WITH PROPER MATH!!!


def explore_model_4_free(turns, da, k_min, k_max, k_samples):
    ks = np.linspace(k_min, k_max, k_samples)
    pars = []
    errs = []
    co_pars = []
    for k in tqdm(ks):
        par, co_par = curve_fit(
            lambda x, a, b : model_4_free(x, a, b, k),
            turns,
            da,
            bounds=([0.01, 0.01], [np.inf, turns[-1] - 0.1])
        )
        pars.append(par)
        co_pars.append(co_par)
        errs.append(chi_2(da, model_4_free(turn_sampling, par[0], par[1], k)))
    return np.asarray(pars), np.asarray(errs), np.asarray(co_pars)

## Load data and setup original DA

In [9]:
savepath = "data/"

sigma = 0.2
turn_samples = 100
skipper = 1

cut_point = 1.0

### Loading Hénon data and computing losses (this takes a long time)

In [10]:
epsilons = []

real_losses = []
uniform_losses = []
gaussian_losses = []

real_DAs = []
uniform_DAs = []
gaussian_DAs = []

files = list(sorted(list(filter(lambda f: "henon_eps_" in f, os.listdir(savepath))), key=lambda f: float(f[10: -4])))[::skipper]

for f in tqdm(files):
    print("Loading file: ", f)
    epsilon = float(f[10: -4])
    epsilons.append(epsilon)
    engine = sx.uniform_radial_scanner.load_values(savepath + f)

    baseline_samples = engine.baseline_samples
    min_turns = 10**3
    max_turns = 10**6
    d_r = engine.dr
    turn_sampling = np.linspace(min_turns, max_turns, turn_samples, dtype=np.int)[::-1]
    turn_sampling_with_zero = np.concatenate((turn_sampling, [0.0]))
    
    print("Real part")
    real_DAs.append(engine.compute_DA_standard(turn_sampling))
    print("Done DA")
    
    print("Uniform part")
    engine.assign_weights(sx.assign_uniform_distribution())
    uniform_losses.append(engine.compute_loss(turn_sampling, cut_point))
    uniform_DAs.append(DA_from_unifom_loss(uniform_losses[-1], cut_point))
    
    print("Gaussian part")
    engine.assign_weights(sx.assign_symmetric_gaussian(sigma))
    gaussian_losses.append(engine.compute_loss(turn_sampling, cut_point))
    gaussian_DAs.append(DA_from_symmetric_gaussian_loss(gaussian_losses[-1], sigma, cut_point))
    


Loading file:  henon_eps_1.pkl
Real part
Done DA
Uniform part
Gaussian part
Loading file:  henon_eps_2.pkl
Real part
Done DA
Uniform part
Gaussian part
Loading file:  henon_eps_4.pkl
Real part
Done DA
Uniform part
Gaussian part
Loading file:  henon_eps_6.pkl
Real part
Done DA
Uniform part
Gaussian part
Loading file:  henon_eps_8.pkl
Real part
Done DA
Uniform part
Gaussian part
Loading file:  henon_eps_10.pkl
Real part
Done DA
Uniform part
Gaussian part
Loading file:  henon_eps_12.pkl
Real part
Done DA
Uniform part
Gaussian part
Loading file:  henon_eps_14.pkl
Real part
Done DA
Uniform part
Gaussian part
Loading file:  henon_eps_16.pkl
Real part
Done DA
Uniform part
Gaussian part
Loading file:  henon_eps_18.pkl
Real part
Done DA
Uniform part
Gaussian part
Loading file:  henon_eps_20.pkl
Real part
Done DA
Uniform part
Gaussian part
Loading file:  henon_eps_22.pkl
Real part
Done DA
Uniform part
Gaussian part
Loading file:  henon_eps_24.pkl
Real part
Done DA
Uniform part
Gaussian part
Load

#### A little visualization

In [11]:
tfig, taxs = plt.subplots()

for i in range(len(real_DAs)):
    taxs.plot(turn_sampling, real_DAs[i], c="C0")
    taxs.plot(turn_sampling, uniform_DAs[i], c="C1")
    taxs.plot(turn_sampling, gaussian_DAs[i], c="C2")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# General model fittings

In [13]:
labels = (
    ("epsilon", ""),
    ("type", ""),
    ("sigma", ""),
    ("Model 2", "k"),
    ("Model 2", "k err"),
    ("Model 2", "rho"),
    ("Model 2", "rho err"),
    ("Model 2", "N0"),
    ("Model 2", "N0 err"),
    ("Model 2", "Chi2"),
    ("Model 4 (2 free pars)", "k"),
    ("Model 4 (2 free pars)", "k err"),
    ("Model 4 (2 free pars)", "rho"),
    ("Model 4 (2 free pars)", "rho err"),
    ("Model 4 (2 free pars)", "lambda"),
    ("Model 4 (2 free pars)", "Chi2"),
    ("Model 4 (3 free pars)", "k"),
    ("Model 4 (3 free pars)", "k err"),
    ("Model 4 (3 free pars)", "rho"),
    ("Model 4 (3 free pars)", "rho err"),
    ("Model 4 (3 free pars)", "N0"),
    ("Model 4 (3 free pars)", "N0 err"),
    ("Model 4 (3 free pars)", "lambda"),
    ("Model 4 (3 free pars)", "Chi2"),
)

fitting_data = pd.DataFrame(columns=pd.MultiIndex.from_tuples(labels))
lost_table = []
lost_table_with_zero = []

In [14]:
k_min = 0.1
k_max = 2.00
samples = 100

ks = np.linspace(k_min, k_max, samples)

In [15]:
for i, eps in tqdm(list(enumerate(epsilons))):
    # Model 2
    real_pars, real_errs, real_co_pars = explore_k_model_2(
        turn_sampling, real_DAs[i], k_min, k_max, samples
    )
    uniform_loss_pars, uniform_loss_errs, uniform_loss_co_pars = explore_k_model_2(
        turn_sampling, uniform_DAs[i], k_min, k_max, samples
    )
    gaussian_loss_pars, gaussian_loss_errs, gaussian_loss_co_pars = explore_k_model_2(
        turn_sampling, gaussian_DAs[i], k_min, k_max, samples
    )

    real_selected_err_2 = np.min(real_errs)
    real_selected_k_2 = ks[np.argmin(real_errs)]
    real_selected_pars_2 = real_pars[np.argmin(real_errs)]
    real_selected_co_pars_2 = real_co_pars[np.argmin(real_errs)]

    uniform_loss_selected_err_2 = np.min(uniform_loss_errs)
    uniform_loss_selected_k_2 = ks[np.argmin(uniform_loss_errs)]
    uniform_loss_selected_pars_2 = uniform_loss_pars[np.argmin(uniform_loss_errs)]
    uniform_loss_selected_co_pars_2 = uniform_loss_co_pars[np.argmin(uniform_loss_errs)]
    
    gaussian_loss_selected_err_2 = np.min(gaussian_loss_errs)
    gaussian_loss_selected_k_2 = ks[np.argmin(gaussian_loss_errs)]
    gaussian_loss_selected_pars_2 = gaussian_loss_pars[np.argmin(gaussian_loss_errs)]
    gaussian_loss_selected_co_pars_2 = gaussian_loss_co_pars[np.argmin(gaussian_loss_errs)]
    
    # Model 4 (2 pars)
    real_pars, real_errs, real_co_pars = explore_k_model_4(
        turn_sampling, real_DAs[i], k_min, k_max, samples
    )
    uniform_loss_pars, uniform_loss_errs, uniform_loss_co_pars = explore_k_model_4(
        turn_sampling, uniform_DAs[i], k_min, k_max, samples
    )
    gaussian_loss_pars, gaussian_loss_errs, gaussian_loss_co_pars = explore_k_model_4(
        turn_sampling, gaussian_DAs[i], k_min, k_max, samples
    )

    real_selected_err_4 = np.min(real_errs)
    real_selected_k_4 = ks[np.argmin(real_errs)]
    real_selected_pars_4 = real_pars[np.argmin(real_errs)]
    real_selected_co_pars_4 = real_co_pars[np.argmin(real_errs)]

    uniform_loss_selected_err_4 = np.min(uniform_loss_errs)
    uniform_loss_selected_k_4 = ks[np.argmin(uniform_loss_errs)]
    uniform_loss_selected_pars_4 = uniform_loss_pars[np.argmin(uniform_loss_errs)]
    uniform_loss_selected_co_pars_4 = uniform_loss_co_pars[np.argmin(uniform_loss_errs)]
    
    gaussian_loss_selected_err_4 = np.min(gaussian_loss_errs)
    gaussian_loss_selected_k_4 = ks[np.argmin(gaussian_loss_errs)]
    gaussian_loss_selected_pars_4 = gaussian_loss_pars[np.argmin(gaussian_loss_errs)]
    gaussian_loss_selected_co_pars_4 = gaussian_loss_co_pars[np.argmin(gaussian_loss_errs)]
    
    # Model 4 (3 pars)
    real_pars, real_errs, real_co_pars = explore_model_4_free(
        turn_sampling, real_DAs[i], k_min, k_max, samples
    )
    uniform_loss_pars, uniform_loss_errs, uniform_loss_co_pars = explore_model_4_free(
        turn_sampling, uniform_DAs[i], k_min, k_max, samples
    )
    gaussian_loss_pars, gaussian_loss_errs, gaussian_loss_co_pars = explore_model_4_free(
        turn_sampling, gaussian_DAs[i], k_min, k_max, samples
    )

    real_selected_err_4_free = np.min(real_errs)
    real_selected_k_4_free = ks[np.argmin(real_errs)]
    real_selected_pars_4_free = real_pars[np.argmin(real_errs)]
    real_selected_co_pars_4_free = real_co_pars[np.argmin(real_errs)]

    uniform_loss_selected_err_4_free = np.min(uniform_loss_errs)
    uniform_loss_selected_k_4_free = ks[np.argmin(uniform_loss_errs)]
    uniform_loss_selected_pars_4_free = uniform_loss_pars[np.argmin(uniform_loss_errs)]
    uniform_loss_selected_co_pars_4_free = uniform_loss_co_pars[np.argmin(uniform_loss_errs)]
    
    gaussian_loss_selected_err_4_free = np.min(gaussian_loss_errs)
    gaussian_loss_selected_k_4_free = ks[np.argmin(gaussian_loss_errs)]
    gaussian_loss_selected_pars_4_free = gaussian_loss_pars[np.argmin(gaussian_loss_errs)]
    gaussian_loss_selected_co_pars_4_free = gaussian_loss_co_pars[np.argmin(gaussian_loss_errs)]
    
    # Saving Data
    fitting_data.loc[len(fitting_data)] = [
        epsilons[i],
        "real",
        np.nan,
        real_selected_k_2,
        ks[1] - ks[0],
        real_selected_pars_2[0],
        np.sqrt(real_selected_co_pars_2[0][0]),
        real_selected_pars_2[1],
        np.sqrt(real_selected_co_pars_2[1][1]),
        real_selected_err_2,
        real_selected_k_4,
        ks[1] - ks[0],
        real_selected_pars_4[0],
        np.sqrt(real_selected_co_pars_4[0][0]),
        1/2,
        real_selected_err_4,
        real_selected_k_4_free,
        ks[1] - ks[0],
        real_selected_pars_4_free[0],
        np.sqrt(real_selected_co_pars_4_free[0][0]),
        real_selected_pars_4_free[1],
        np.sqrt(real_selected_co_pars_4_free[1][1]),
        1/2,
        real_selected_err_4_free
    ]
    
    # lost_table.append(gaussian_losses[i])
    
    fitting_data.loc[len(fitting_data)] = [
        epsilons[i],
        "uniform",
        np.nan,
        uniform_loss_selected_k_2,
        ks[1] - ks[0],
        uniform_loss_selected_pars_2[0],
        np.sqrt(uniform_loss_selected_co_pars_2[0][0]),
        uniform_loss_selected_pars_2[1],
        np.sqrt(uniform_loss_selected_co_pars_2[1][1]),
        uniform_loss_selected_err_2,
        uniform_loss_selected_k_4,
        ks[1] - ks[0],
        uniform_loss_selected_pars_4[0],
        np.sqrt(uniform_loss_selected_co_pars_4[0][0]),
        1/2,
        uniform_loss_selected_err_4,
        uniform_loss_selected_k_4_free,
        ks[1] - ks[0],
        uniform_loss_selected_pars_4_free[0],
        np.sqrt(uniform_loss_selected_co_pars_4_free[0][0]),
        uniform_loss_selected_pars_4_free[1],
        np.sqrt(uniform_loss_selected_co_pars_4_free[1][1]),
        1/2,
        uniform_loss_selected_err_4_free
    ]
    
    # lost_table.append(uniform_losses[i])
    
    fitting_data.loc[len(fitting_data)] = [
        epsilons[i],
        "gaussian",
        sigma,
        gaussian_loss_selected_k_2,
        ks[1] - ks[0],
        gaussian_loss_selected_pars_2[0],
        np.sqrt(gaussian_loss_selected_co_pars_2[0][0]),
        gaussian_loss_selected_pars_2[1],
        np.sqrt(gaussian_loss_selected_co_pars_2[1][1]),
        gaussian_loss_selected_err_2,
        gaussian_loss_selected_k_4,
        ks[1] - ks[0],
        gaussian_loss_selected_pars_4[0],
        np.sqrt(gaussian_loss_selected_co_pars_4[0][0]),
        1/2,
        gaussian_loss_selected_err_4,
        gaussian_loss_selected_k_4_free,
        ks[1] - ks[0],
        gaussian_loss_selected_pars_4_free[0],
        np.sqrt(gaussian_loss_selected_co_pars_4_free[0][0]),
        gaussian_loss_selected_pars_4_free[1],
        np.sqrt(gaussian_loss_selected_co_pars_4_free[1][1]),
        1/2,
        gaussian_loss_selected_err_4_free
    ]
    
    lost_table.append(gaussian_losses[i])
    lost_table_with_zero.append(np.concatenate((gaussian_losses[i], [1.0])))

#### Just an example plot

In [16]:
tfig, taxs = plt.subplots()

taxs.plot(turn_sampling, real_DAs[i], label="Data")
taxs.plot(turn_sampling, model_2(turn_sampling, real_selected_pars_2[0], real_selected_pars_2[1], real_selected_k_2), label="Model 2")
taxs.plot(turn_sampling, model_4(turn_sampling, real_selected_pars_4[0], real_selected_k_4), label="Model 4 (2 pars)")
taxs.plot(turn_sampling, model_4_free(turn_sampling, real_selected_pars_4_free[0], real_selected_pars_4_free[1], real_selected_k_4_free), label="Model 4 (3 pars)")
taxs.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Visualizing dataframes and parameters

In [17]:
fitting_data[["epsilon", "type", "sigma", "Model 2"]]

epsilon      type sigma   Model 2                                 \
                                   k     k err        rho   rho err   
0      1.0      real   NaN  0.157576  0.019192   1.238926  0.000845   
1      1.0   uniform   NaN  0.119192  0.019192   1.030356  0.000167   
2      1.0  gaussian   0.2  0.119192  0.019192   0.994001  0.000145   
3      2.0      real   NaN  0.234343  0.019192   1.835630  0.001863   
4      2.0   uniform   NaN  0.138384  0.019192   1.131076  0.000243   
5      2.0  gaussian   0.2  0.157576  0.019192   1.205789  0.000482   
6      4.0      real   NaN  0.330303  0.019192   2.950729  0.002771   
7      4.0   uniform   NaN  0.176768  0.019192   1.359592  0.000469   
8      4.0  gaussian   0.2  0.215152  0.019192   1.596389  0.001198   
9      6.0      real   NaN  0.368687  0.019192   3.509941  0.002510   
10     6.0   uniform   NaN  0.215152  0.019192   1.631702  0.000885   
11     6.0  gaussian   0.2  0.291919  0.019192   2.325704  0.002355   
12     8.0      real   NaN  0.387879  0.019192   3.800330  0.001897   
13     8.0   uniform   NaN  0.253535  0.019192   1.958189  0.001169   
14     8.0  gaussian   0.2  0.330303  0.019192   2.783196  0.002517   
15    10.0      real   NaN  0.483838  0.019192   6.072436  0.004085   
16    10.0   uniform   NaN  0.272727  0.019192   2.133858  0.001081   
17    10.0  gaussian   0.2  0.368687  0.019192   3.334442  0.002629   
18    12.0      real   NaN  0.579798  0.019192   9.658579  0.009915   
19    12.0   uniform   NaN  0.291919  0.019192   2.326492  0.001182   
20    12.0  gaussian   0.2  0.368687  0.019192   3.296056  0.001971   
21    14.0      real   NaN  0.656566  0.019192  13.928873  0.016458   
22    14.0   uniform   NaN  0.311111  0.019192   2.537171  0.001251   
23    14.0  gaussian   0.2  0.387879  0.019192   3.596388  0.001804   
24    16.0      real   NaN  0.694949  0.019192  16.553934  0.019473   
25    16.0   uniform   NaN  0.311111  0.019192   2.516682  0.001069   
26    16.0  gaussian   0.2  0.387879  0.019192   3.562639  0.001439   
27    18.0      real   NaN  0.771717  0.019192  23.740494  0.036355   
28    18.0   uniform   NaN  0.330303  0.019192   2.745081  0.000939   
29    18.0  gaussian   0.2  0.387879  0.019192   3.531043  0.000982   
30    20.0      real   NaN  0.829293  0.019192  30.852175  0.058613   
31    20.0   uniform   NaN  0.330303  0.019192   2.728318  0.000777   
32    20.0  gaussian   0.2  0.387879  0.019192   3.505556  0.000851   
33    22.0      real   NaN  0.867677  0.019192  36.195525  0.095800   
34    22.0   uniform   NaN  0.349495  0.019192   2.976359  0.001073   
35    22.0  gaussian   0.2  0.407071  0.019192   3.826263  0.001002   
36    24.0      real   NaN  0.886869  0.019192  38.663000  0.100705   
37    24.0   uniform   NaN  0.349495  0.019192   2.957866  0.000865   
38    24.0  gaussian   0.2  0.407071  0.019192   3.794873  0.001169   
39    26.0      real   NaN  0.906061  0.019192  41.431612  0.116709   
40    26.0   uniform   NaN  0.368687  0.019192   3.226670  0.001421   
41    26.0  gaussian   0.2  0.426263  0.019192   4.133348  0.001783   
42    28.0      real   NaN  0.944444  0.019192  48.406836  0.147140   
43    28.0   uniform   NaN  0.387879  0.019192   3.513950  0.002353   
44    28.0  gaussian   0.2  0.445455  0.019192   4.497562  0.002671   
45    30.0      real   NaN  0.944444  0.019192  47.853141  0.125729   
46    30.0   uniform   NaN  0.407071  0.019192   3.823105  0.003075   
47    30.0  gaussian   0.2  0.483838  0.019192   5.376352  0.004248   

                                        
            N0        N0 err      Chi2  
0    39.275027  1.458306e+00  0.000119  
1   294.183743  2.272692e+00  0.000030  
2   227.810731  1.725574e+00  0.000018  
3     3.913099  1.875596e-01  0.000110  
4   202.910975  2.000384e+00  0.000038  
5    88.876988  1.705396e+00  0.000067  
6     0.327927  1.269758e-02  0.000050  
7    94.761857  1.379030e+00  0.000056  
8    24.374490  7.713151e-01  0.000121  
9

In [19]:
fitting_data[["epsilon", "type", "sigma", "Model 4 (2 free pars)"]]

epsilon      type sigma Model 4 (2 free pars)                      \
                                               k     k err       rho   
0      1.0      real   NaN              0.272727  0.019192  2.333930   
1      1.0   uniform   NaN              0.311111  0.019192  2.875817   
2      1.0  gaussian   0.2              0.291919  0.019192  2.532688   
3      2.0      real   NaN              0.291919  0.019192  2.528580   
4      2.0   uniform   NaN              0.330303  0.019192  3.121093   
5      2.0  gaussian   0.2              0.311111  0.019192  2.749120   
6      4.0      real   NaN              0.330303  0.019192  2.969620   
7      4.0   uniform   NaN              0.349495  0.019192  3.370596   
8      4.0  gaussian   0.2              0.349495  0.019192  3.230869   
9      6.0      real   NaN              0.349495  0.019192  3.203132   
10     6.0   uniform   NaN              0.368687  0.019192  3.640398   
11     6.0  gaussian   0.2              0.368687  0.019192  3.485795   
12     8.0      real   NaN              0.368687  0.019192  3.458538   
13     8.0   uniform   NaN              0.387879  0.019192  3.931041   
14     8.0  gaussian   0.2              0.387879  0.019192  3.763178   
15    10.0      real   NaN              0.387879  0.019192  3.729218   
16    10.0   uniform   NaN              0.407071  0.019192  4.245400   
17    10.0  gaussian   0.2              0.407071  0.019192  4.061896   
18    12.0      real   NaN              0.407071  0.019192  4.021888   
19    12.0   uniform   NaN              0.426263  0.019192  4.581564   
20    12.0  gaussian   0.2              0.407071  0.019192  4.044215   
21    14.0      real   NaN              0.407071  0.019192  3.999251   
22    14.0   uniform   NaN              0.426263  0.019192  4.563098   
23    14.0  gaussian   0.2              0.426263  0.019192  4.367939   
24    16.0      real   NaN              0.426263  0.019192  4.314201   
25    16.0   uniform   NaN              0.426263  0.019192  4.545620   
26    16.0  gaussian   0.2              0.426263  0.019192  4.351537   
27    18.0      real   NaN              0.445455  0.019192  4.648181   
28    18.0   uniform   NaN              0.445455  0.019192  4.905066   
29    18.0  gaussian   0.2              0.426263  0.019192  4.334964   
30    20.0      real   NaN              0.445455  0.019192  4.624434   
31    20.0   uniform   NaN              0.445455  0.019192  4.891625   
32    20.0  gaussian   0.2              0.445455  0.019192  4.680148   
33    22.0      real   NaN              0.464646  0.019192  4.977479   
34    22.0   uniform   NaN              0.464646  0.019192  5.275160   
35    22.0  gaussian   0.2              0.445455  0.019192  4.664422   
36    24.0      real   NaN              0.483838  0.019192  5.352301   
37    24.0   uniform   NaN              0.464646  0.019192  5.259068   
38    24.0  gaussian   0.2              0.445455  0.019192  4.648330   
39    26.0      real   NaN              0.483838  0.019192  5.325588   
40    26.0   uniform   NaN              0.464646  0.019192  5.243864   
41    26.0  gaussian   0.2              0.464646  0.019192  5.012674   
42    28.0      real   NaN              0.503030  0.019192  5.725366   
43    28.0   uniform   NaN              0.483838  0.019192  5.651677   
44    28.0  gaussian   0.2              0.464646  0.019192  4.993789   
45    30.0      real   NaN              0.503030  0.019192  5.698385   
46    30.0   uniform   NaN              0.483838  0.019192  5.630003   
47    30.0  gaussian   0.2              0.464646  0.019192  4.975234   

                               
     rho err lambda      Chi2  
0   0.000596    0.5  0.000299  
1   0.001857    0.5  0.001967  
2   0.001271    0.5  0.001145  
3   0.000488    0.5  0.000167  
4   0.001761    0.5  0.001500  
5   0.001092    0.5  0.000703  
6   0.000314    0.5  0.000052  
7   0.001498    0.5  0.000885  
8   0.000861    0.5  0.000319  
9   0.000280    0.5  0.000036  
10  0.001313    0.

In [20]:
fitting_data[["epsilon", "type", "sigma", "Model 4 (3 free pars)"]]

epsilon      type sigma Model 4 (3 free pars)                       \
                                               k     k err        rho   
0      1.0      real   NaN              0.157576  0.019192   1.241399   
1      1.0   uniform   NaN              0.119192  0.019192   1.032455   
2      1.0  gaussian   0.2              0.119192  0.019192   0.995834   
3      2.0      real   NaN              0.234343  0.019192   1.841246   
4      2.0   uniform   NaN              0.138384  0.019192   1.133767   
5      2.0  gaussian   0.2              0.157576  0.019192   1.208683   
6      4.0      real   NaN              0.330303  0.019192   2.964484   
7      4.0   uniform   NaN              0.176768  0.019192   1.363790   
8      4.0  gaussian   0.2              0.234343  0.019192   1.769843   
9      6.0      real   NaN              0.368687  0.019192   3.529348   
10     6.0   uniform   NaN              0.215152  0.019192   1.637964   
11     6.0  gaussian   0.2              0.291919  0.019192   2.336758   
12     8.0      real   NaN              0.407071  0.019192   4.223207   
13     8.0   uniform   NaN              0.253535  0.019192   1.967140   
14     8.0  gaussian   0.2              0.349495  0.019192   3.087300   
15    10.0      real   NaN              0.483838  0.019192   6.120074   
16    10.0   uniform   NaN              0.272727  0.019192   2.144629   
17    10.0  gaussian   0.2              0.368687  0.019192   3.355687   
18    12.0      real   NaN              0.522222  0.019192   7.270618   
19    12.0   uniform   NaN              0.291919  0.019192   2.339293   
20    12.0  gaussian   0.2              0.387879  0.019192   3.656028   
21    14.0      real   NaN              0.541414  0.019192   7.856494   
22    14.0   uniform   NaN              0.311111  0.019192   2.552297   
23    14.0  gaussian   0.2              0.387879  0.019192   3.621543   
24    16.0      real   NaN              0.560606  0.019192   8.507998   
25    16.0   uniform   NaN              0.330303  0.019192   2.785776   
26    16.0  gaussian   0.2              0.407071  0.019192   3.951383   
27    18.0      real   NaN              0.579798  0.019192   9.189829   
28    18.0   uniform   NaN              0.330303  0.019192   2.763043   
29    18.0  gaussian   0.2              0.407071  0.019192   3.915569   
30    20.0      real   NaN              0.598990  0.019192   9.926628   
31    20.0   uniform   NaN              0.330303  0.019192   2.746466   
32    20.0  gaussian   0.2              0.407071  0.019192   3.886724   
33    22.0      real   NaN              0.618182  0.019192  10.682352   
34    22.0   uniform   NaN              0.349495  0.019192   2.997483   
35    22.0  gaussian   0.2              0.407071  0.019192   3.856132   
36    24.0      real   NaN              0.637374  0.019192  11.501512   
37    24.0   uniform   NaN              0.349495  0.019192   2.979144   
38    24.0  gaussian   0.2              0.426263  0.019192   4.207731   
39    26.0      real   NaN              0.637374  0.019192  11.295762   
40    26.0   uniform   NaN              0.368687  0.019192   3.251382   
41    26.0  gaussian   0.2              0.445455  0.019192   4.583939   
42    28.0      real   NaN              0.637374  0.019192  11.109468   
43    28.0   uniform   NaN              0.387879  0.019192   3.542667   
44    28.0  gaussian   0.2              0.464646  0.019192   4.988860   
45    30.0      real   NaN              0.637374  0.019192  10.953281   
46    30.0   uniform   NaN              0.407071  0.019192   3.856344   
47    30.0  gaussian   0.2              0.483838  0.019192   5.429142   

                                                     
     rho err          N0    N0 err lambda      Chi2  
0   0.000832   41.643311  1.521502    0.5  0.000120  
1   0.000162  292.866531  2.176750    0.5  0.000030  
2   0.000150  228.978179  1.793332    0.5  0.000021  
3   0.001831    4.485467  0.210999    0.5  0.000110  
4   0.000235  205.086037  1.

In [21]:
fitting_data.loc[fitting_data[("type", "")]=="gaussian"][["epsilon", "type", "sigma", "Model 4 (3 free pars)"]]

epsilon      type sigma Model 4 (3 free pars)                      \
                                               k     k err       rho   
2      1.0  gaussian   0.2              0.119192  0.019192  0.995834   
5      2.0  gaussian   0.2              0.157576  0.019192  1.208683   
8      4.0  gaussian   0.2              0.234343  0.019192  1.769843   
11     6.0  gaussian   0.2              0.291919  0.019192  2.336758   
14     8.0  gaussian   0.2              0.349495  0.019192  3.087300   
17    10.0  gaussian   0.2              0.368687  0.019192  3.355687   
20    12.0  gaussian   0.2              0.387879  0.019192  3.656028   
23    14.0  gaussian   0.2              0.387879  0.019192  3.621543   
26    16.0  gaussian   0.2              0.407071  0.019192  3.951383   
29    18.0  gaussian   0.2              0.407071  0.019192  3.915569   
32    20.0  gaussian   0.2              0.407071  0.019192  3.886724   
35    22.0  gaussian   0.2              0.407071  0.019192  3.856132   
38    24.0  gaussian   0.2              0.426263  0.019192  4.207731   
41    26.0  gaussian   0.2              0.445455  0.019192  4.583939   
44    28.0  gaussian   0.2              0.464646  0.019192  4.988860   
47    30.0  gaussian   0.2              0.483838  0.019192  5.429142   

                                                     
     rho err          N0    N0 err lambda      Chi2  
2   0.000150  228.978179  1.793332    0.5  0.000021  
5   0.000461   92.528556  1.698230    0.5  0.000064  
8   0.001463   15.092161  0.519179    0.5  0.000119  
11  0.002309    4.516531  0.169536    0.5  0.000112  
14  0.002943    1.300153  0.044136    0.5  0.000072  
17  0.002556    1.053307  0.027597    0.5  0.000043  
20  0.002264    0.803063  0.016638    0.5  0.000027  
23  0.001759    0.969323  0.015504    0.5  0.000017  
26  0.001679    0.709426  0.009711    0.5  0.000012  
29  0.001128    0.839783  0.007685    0.5  0.000006  
32  0.000952    0.954613  0.007353    0.5  0.000004  
35  0.000950    1.099469  0.008417    0.5  0.000005  
38  0.001411    0.798403  0.008164    0.5  0.000008  
41  0.002093    0.602675  0.008220    0.5  0.000014  
44  0.003025    0.464120  0.008233    0.5  0.000024  
47  0.004119    0.356817  0.007766    0.5  0.000035

In [44]:
fig1, ax1 = plt.subplots(1,3)

p_filter = fitting_data[("type", "")] == "real" 

ax1[0].scatter(
    fitting_data[p_filter][("epsilon", "")],
    fitting_data[p_filter][("Model 2", "k")],
    label="Model 2"
)
ax1[0].scatter(
    fitting_data[p_filter][("epsilon", "")],
    fitting_data[p_filter][("Model 4 (2 free pars)", "k")],
    label="Model 4 (2 pars)"
)
ax1[0].scatter(
    fitting_data[p_filter][("epsilon", "")],
    fitting_data[p_filter][("Model 4 (3 free pars)", "k")],
    label="Model 4 (3 pars)"
)

ax1[0].set_xlabel("$\\epsilon$")
ax1[0].set_ylabel("$\\kappa$")
ax1[0].set_title("Real DA")
ax1[0].legend(fontsize="xx-small")

p_filter = fitting_data[("type", "")] == "uniform" 

ax1[1].scatter(
    fitting_data[p_filter][("epsilon", "")],
    fitting_data[p_filter][("Model 2", "k")],
    label="Model 2"
)
ax1[1].scatter(
    fitting_data[p_filter][("epsilon", "")],
    fitting_data[p_filter][("Model 4 (2 free pars)", "k")],
    label="Model 4 (2 pars)"
)
ax1[1].scatter(
    fitting_data[p_filter][("epsilon", "")],
    fitting_data[p_filter][("Model 4 (3 free pars)", "k")],
    label="Model 4 (3 pars)"
)

ax1[1].set_xlabel("$\\epsilon$")
ax1[1].set_ylabel("$\\kappa$")
ax1[1].set_title("DA from Uniform loss")
ax1[1].legend(fontsize="xx-small")

p_filter = fitting_data[("type", "")] == "gaussian" 

ax1[2].scatter(
    fitting_data[p_filter][("epsilon", "")],
    fitting_data[p_filter][("Model 2", "k")],
    label="Model 2"
)
ax1[2].scatter(
    fitting_data[p_filter][("epsilon", "")],
    fitting_data[p_filter][("Model 4 (2 free pars)", "k")],
    label="Model 4 (2 pars)"
)
ax1[2].scatter(
    fitting_data[p_filter][("epsilon", "")],
    fitting_data[p_filter][("Model 4 (3 free pars)", "k")],
    label="Model 4 (3 pars)"
)
ax1[2].set_xlabel("$\\epsilon$")
ax1[2].set_ylabel("$\\kappa$")
ax1[2].set_title("DA from Gaussian loss")
ax1[2].legend(fontsize="xx-small")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# FP process (fixed K)

### Setting up dataframe

In [22]:
labels = (
    ("epsilon", ""),
    ("type", ""),
    ("sigma", ""),
    ("Fokker-Planck", "I_star"),
    ("Fokker-Planck", "time_scale")
)

fp_data = []
fp_fit_data = pd.DataFrame(columns=pd.MultiIndex.from_tuples(labels))

## Computing

In [23]:
selected_data = fitting_data.loc[fitting_data[("type", "")]=="gaussian"]

for i in tqdm(range(len(selected_data[("Model 4 (3 free pars)", "k")]))):
    k = selected_data[("Model 4 (3 free pars)", "k")].iloc[i]
    rho = selected_data[("Model 4 (3 free pars)", "rho")].iloc[i]
    N0 = selected_data[("Model 4 (3 free pars)", "N0")].iloc[i]
    
    I_max = cut_point**2 / 2
    I = np.linspace(0, I_max, 1000)
    I0 = I * np.exp(-(I/sigma**2))
    I0 /= integrate.trapz(I0, I)

    c = 1.0

    I_star_min = 0.01 ** 2
    I_star_max = 0.5 ** 2
    I_star_steps = 101
    I_star_list = np.linspace(I_star_min, I_star_max, I_star_steps)

    iter_step = 10

    data = []
    errors = []
    original_times = []

    dt = 0.0001
    for j, I_star in tqdm(enumerate(I_star_list), total=I_star_steps):
        engine_nk = nk.cn_nekhoroshev(I_max, c, I_star, 1 / (k * 2), 0, I0, dt)
        step = 0

        t = []
        survival = []

        while(1.0 - engine_nk.get_particle_loss() >= lost_table_with_zero[i][0]):
            t.append(step * iter_step * dt)
            survival.append(1.0 - engine_nk.get_particle_loss())
            engine_nk.iterate(iter_step)
            #print("loss:", 1.0 - engine_nk.get_particle_loss())
            step += 1
            if step > 10000:
                break
        t.append(step * iter_step * dt)
        survival.append(1.0 - engine_nk.get_particle_loss())

        print("Step:", j, "/", I_star_steps, ",", "I_star:{:.3}".format(I_star), ",", "Number of steps executed", step, ", dt:", dt)
        if step > 1000:
            print("Increasing dt!")
            dt *= 10

        f = interp1d(t, survival, kind="cubic")
        inv_f = inversefunc(
            f,
            domain=(t[0], t[-1]),
            image=(survival[-1], survival[0])
        )
        point_t = inv_f(lost_table[i][0])
        points_t = np.linspace(0, point_t, len(turn_sampling_with_zero))
        values_f = f(points_t)
        values_f = values_f[::-1]

        data.append(values_f)
        errors.append(chi_2(lost_table_with_zero[i], values_f))
        print("Error:", errors[-1])
        original_times.append(np.asarray(t)[::-1])
        if j > I_star_steps * 0.1:
            if(errors[-2] - errors[-1] < 0):
                print("Found minimum! Breaking!")
                break
                
    best_idx = np.argmin(errors)
    best_data = data[best_idx]
    best_time = original_times[best_idx]
    best_I_star = I_star_list[best_idx]
    
    fp_data.append(best_data)
    fp_fit_data.loc[len(fp_fit_data)] = [
        selected_data[("epsilon", "")].iloc[i],
        selected_data[("type", "")].iloc[i],
        selected_data[("sigma", "")].iloc[i],
        best_I_star,
        best_time
    ]
    

Step: 0 / 101 , I_star:0.0001 , Number of steps executed 83 , dt: 0.0001
Error: 1.9193813893529605
Step: 1 / 101 , I_star:0.0026 , Number of steps executed 83 , dt: 0.0001
Error: 1.9468076259013416
Step: 2 / 101 , I_star:0.0051 , Number of steps executed 83 , dt: 0.0001
Error: 1.9496474329058673
Step: 3 / 101 , I_star:0.0076 , Number of steps executed 83 , dt: 0.0001
Error: 1.9508415064666014
Step: 4 / 101 , I_star:0.0101 , Number of steps executed 83 , dt: 0.0001
Error: 1.9512813339144814
Step: 5 / 101 , I_star:0.0126 , Number of steps executed 83 , dt: 0.0001
Error: 1.9509829964609535
Step: 6 / 101 , I_star:0.0151 , Number of steps executed 83 , dt: 0.0001
Error: 1.949831631862281
Step: 7 / 101 , I_star:0.0176 , Number of steps executed 83 , dt: 0.0001
Error: 1.947916206154713
Step: 8 / 101 , I_star:0.0201 , Number of steps executed 83 , dt: 0.0001
Error: 1.9452582015503639
Step: 9 / 101 , I_star:0.0226 , Number of steps executed 83 , dt: 0.0001
Error: 1.9418754213541713
Step: 10 / 1

Step: 0 / 101 , I_star:0.0001 , Number of steps executed 85 , dt: 0.0001
Error: 1.9714747476554173
Step: 1 / 101 , I_star:0.0026 , Number of steps executed 85 , dt: 0.0001
Error: 1.9992424517253837
Step: 2 / 101 , I_star:0.0051 , Number of steps executed 85 , dt: 0.0001
Error: 2.001478217757254
Step: 3 / 101 , I_star:0.0076 , Number of steps executed 85 , dt: 0.0001
Error: 2.0027894449687413
Step: 4 / 101 , I_star:0.0101 , Number of steps executed 85 , dt: 0.0001
Error: 2.003367880802103
Step: 5 / 101 , I_star:0.0126 , Number of steps executed 85 , dt: 0.0001
Error: 2.003241670569028
Step: 6 / 101 , I_star:0.0151 , Number of steps executed 85 , dt: 0.0001
Error: 2.002405008810723
Step: 7 / 101 , I_star:0.0176 , Number of steps executed 85 , dt: 0.0001
Error: 2.0008627819513363
Step: 8 / 101 , I_star:0.0201 , Number of steps executed 85 , dt: 0.0001
Error: 1.9986263013056123
Step: 9 / 101 , I_star:0.0226 , Number of steps executed 85 , dt: 0.0001
Error: 1.9957088147049054
Step: 10 / 101

Step: 0 / 101 , I_star:0.0001 , Number of steps executed 89 , dt: 0.0001
Error: 2.0954605919954172
Step: 1 / 101 , I_star:0.0026 , Number of steps executed 89 , dt: 0.0001
Error: 2.122557653396265
Step: 2 / 101 , I_star:0.0051 , Number of steps executed 89 , dt: 0.0001
Error: 2.1247657056738363
Step: 3 / 101 , I_star:0.0076 , Number of steps executed 89 , dt: 0.0001
Error: 2.1261596791504433
Step: 4 / 101 , I_star:0.0101 , Number of steps executed 89 , dt: 0.0001
Error: 2.1268061979339907
Step: 5 / 101 , I_star:0.0126 , Number of steps executed 89 , dt: 0.0001
Error: 2.126952647589146
Step: 6 / 101 , I_star:0.0151 , Number of steps executed 89 , dt: 0.0001
Error: 2.1265432843001646
Step: 7 / 101 , I_star:0.0176 , Number of steps executed 89 , dt: 0.0001
Error: 2.125547923726384
Step: 8 / 101 , I_star:0.0201 , Number of steps executed 89 , dt: 0.0001
Error: 2.1239616655599183
Step: 9 / 101 , I_star:0.0226 , Number of steps executed 90 , dt: 0.0001
Error: 2.1217745594240323
Step: 10 / 10

Step: 0 / 101 , I_star:0.0001 , Number of steps executed 92 , dt: 0.0001
Error: 2.2222347626237995
Step: 1 / 101 , I_star:0.0026 , Number of steps executed 92 , dt: 0.0001
Error: 2.250080318468096
Step: 2 / 101 , I_star:0.0051 , Number of steps executed 92 , dt: 0.0001
Error: 2.251499983472628
Step: 3 / 101 , I_star:0.0076 , Number of steps executed 92 , dt: 0.0001
Error: 2.252407670012367
Step: 4 / 101 , I_star:0.0101 , Number of steps executed 92 , dt: 0.0001
Error: 2.2530484992664594
Step: 5 / 101 , I_star:0.0126 , Number of steps executed 92 , dt: 0.0001
Error: 2.253263671494315
Step: 6 / 101 , I_star:0.0151 , Number of steps executed 92 , dt: 0.0001
Error: 2.2530007070221543
Step: 7 / 101 , I_star:0.0176 , Number of steps executed 93 , dt: 0.0001
Error: 2.252233652769734
Step: 8 / 101 , I_star:0.0201 , Number of steps executed 94 , dt: 0.0001
Error: 2.250936217289062
Step: 9 / 101 , I_star:0.0226 , Number of steps executed 94 , dt: 0.0001
Error: 2.2490841532037065
Step: 10 / 101 ,

Step: 0 / 101 , I_star:0.0001 , Number of steps executed 93 , dt: 0.0001
Error: 2.3387410583707413
Step: 1 / 101 , I_star:0.0026 , Number of steps executed 94 , dt: 0.0001
Error: 2.3661923998306404
Step: 2 / 101 , I_star:0.0051 , Number of steps executed 94 , dt: 0.0001
Error: 2.3675666350815714
Step: 3 / 101 , I_star:0.0076 , Number of steps executed 94 , dt: 0.0001
Error: 2.368324864585527
Step: 4 / 101 , I_star:0.0101 , Number of steps executed 95 , dt: 0.0001
Error: 2.3688338764213897
Step: 5 / 101 , I_star:0.0126 , Number of steps executed 96 , dt: 0.0001
Error: 2.3689891830507674
Step: 6 / 101 , I_star:0.0151 , Number of steps executed 96 , dt: 0.0001
Error: 2.368744625468257
Step: 7 / 101 , I_star:0.0176 , Number of steps executed 97 , dt: 0.0001
Error: 2.368081986671207
Step: 8 / 101 , I_star:0.0201 , Number of steps executed 98 , dt: 0.0001
Error: 2.366954148750313
Step: 9 / 101 , I_star:0.0226 , Number of steps executed 99 , dt: 0.0001
Error: 2.3653311850155827
Step: 10 / 101

...

Step: 0 / 101 , I_star:0.0001 , Number of steps executed 103 , dt: 0.0001
Error: 2.942825435597585
Step: 1 / 101 , I_star:0.0026 , Number of steps executed 103 , dt: 0.0001
Error: 2.9722574256183134
Step: 2 / 101 , I_star:0.0051 , Number of steps executed 104 , dt: 0.0001
Error: 2.9737722586859294
Step: 3 / 101 , I_star:0.0076 , Number of steps executed 105 , dt: 0.0001
Error: 2.9751710025281835
Step: 4 / 101 , I_star:0.0101 , Number of steps executed 106 , dt: 0.0001
Error: 2.975718043204458
Step: 5 / 101 , I_star:0.0126 , Number of steps executed 107 , dt: 0.0001
Error: 2.9759383914078055
Step: 6 / 101 , I_star:0.0151 , Number of steps executed 108 , dt: 0.0001
Error: 2.975838398787741
Step: 7 / 101 , I_star:0.0176 , Number of steps executed 110 , dt: 0.0001
Error: 2.975296460604222
Step: 8 / 101 , I_star:0.0201 , Number of steps executed 111 , dt: 0.0001
Error: 2.97426890782084
Step: 9 / 101 , I_star:0.0226 , Number of steps executed 113 , dt: 0.0001
Error: 2.9727168691240817
Step: 

Step: 0 / 101 , I_star:0.0001 , Number of steps executed 104 , dt: 0.0001
Error: 3.0216790490399297
Step: 1 / 101 , I_star:0.0026 , Number of steps executed 105 , dt: 0.0001
Error: 3.051610275643429
Step: 2 / 101 , I_star:0.0051 , Number of steps executed 106 , dt: 0.0001
Error: 3.05265995365881
Step: 3 / 101 , I_star:0.0076 , Number of steps executed 107 , dt: 0.0001
Error: 3.0539107002700434
Step: 4 / 101 , I_star:0.0101 , Number of steps executed 108 , dt: 0.0001
Error: 3.0546666867530172
Step: 5 / 101 , I_star:0.0126 , Number of steps executed 109 , dt: 0.0001
Error: 3.054922622954818
Step: 6 / 101 , I_star:0.0151 , Number of steps executed 111 , dt: 0.0001
Error: 3.054790595577529
Step: 7 / 101 , I_star:0.0176 , Number of steps executed 112 , dt: 0.0001
Error: 3.054259151540015
Step: 8 / 101 , I_star:0.0201 , Number of steps executed 114 , dt: 0.0001
Error: 3.0532704147663527
Step: 9 / 101 , I_star:0.0226 , Number of steps executed 116 , dt: 0.0001
Error: 3.0517769298795194
Step: 

Step: 0 / 101 , I_star:0.0001 , Number of steps executed 105 , dt: 0.0001
Error: 3.1008175243957217
Step: 1 / 101 , I_star:0.0026 , Number of steps executed 106 , dt: 0.0001
Error: 3.131208911583458
Step: 2 / 101 , I_star:0.0051 , Number of steps executed 108 , dt: 0.0001
Error: 3.132315439813213
Step: 3 / 101 , I_star:0.0076 , Number of steps executed 109 , dt: 0.0001
Error: 3.1332121859702275
Step: 4 / 101 , I_star:0.0101 , Number of steps executed 110 , dt: 0.0001
Error: 3.1338743781045957
Step: 5 / 101 , I_star:0.0126 , Number of steps executed 112 , dt: 0.0001
Error: 3.134173168043805
Step: 6 / 101 , I_star:0.0151 , Number of steps executed 114 , dt: 0.0001
Error: 3.134074692402355
Step: 7 / 101 , I_star:0.0176 , Number of steps executed 115 , dt: 0.0001
Error: 3.1335625718684406
Step: 8 / 101 , I_star:0.0201 , Number of steps executed 117 , dt: 0.0001
Error: 3.1326060331429053
Step: 9 / 101 , I_star:0.0226 , Number of steps executed 119 , dt: 0.0001
Error: 3.131166867684026
Step:

Step: 0 / 101 , I_star:0.0001 , Number of steps executed 107 , dt: 0.0001
Error: 3.181119125574354
Step: 1 / 101 , I_star:0.0026 , Number of steps executed 108 , dt: 0.0001
Error: 3.21196519874605
Step: 2 / 101 , I_star:0.0051 , Number of steps executed 110 , dt: 0.0001
Error: 3.213279848438899
Step: 3 / 101 , I_star:0.0076 , Number of steps executed 111 , dt: 0.0001
Error: 3.2140492995443766
Step: 4 / 101 , I_star:0.0101 , Number of steps executed 113 , dt: 0.0001
Error: 3.21451751605946
Step: 5 / 101 , I_star:0.0126 , Number of steps executed 115 , dt: 0.0001
Error: 3.2147157186072435
Step: 6 / 101 , I_star:0.0151 , Number of steps executed 116 , dt: 0.0001
Error: 3.214590730816399
Step: 7 / 101 , I_star:0.0176 , Number of steps executed 118 , dt: 0.0001
Error: 3.2140891422373725
Step: 8 / 101 , I_star:0.0201 , Number of steps executed 120 , dt: 0.0001
Error: 3.2131650727267402
Step: 9 / 101 , I_star:0.0226 , Number of steps executed 122 , dt: 0.0001
Error: 3.211778687018511
Step: 10

Step: 0 / 101 , I_star:0.0001 , Number of steps executed 108 , dt: 0.0001
Error: 3.2642063506317913
Step: 1 / 101 , I_star:0.0026 , Number of steps executed 110 , dt: 0.0001
Error: 3.2955332271870277
Step: 2 / 101 , I_star:0.0051 , Number of steps executed 112 , dt: 0.0001
Error: 3.29697518554644
Step: 3 / 101 , I_star:0.0076 , Number of steps executed 114 , dt: 0.0001
Error: 3.2976126891391493
Step: 4 / 101 , I_star:0.0101 , Number of steps executed 115 , dt: 0.0001
Error: 3.297967969770068
Step: 5 / 101 , I_star:0.0126 , Number of steps executed 117 , dt: 0.0001
Error: 3.298106133827591
Step: 6 / 101 , I_star:0.0151 , Number of steps executed 119 , dt: 0.0001
Error: 3.297954163932131
Step: 7 / 101 , I_star:0.0176 , Number of steps executed 122 , dt: 0.0001
Error: 3.2974502380627975
Step: 8 / 101 , I_star:0.0201 , Number of steps executed 124 , dt: 0.0001
Error: 3.2965469045924536
Step: 9 / 101 , I_star:0.0226 , Number of steps executed 126 , dt: 0.0001
Error: 3.295204669370413
Step: 

## Visualizing dataframe and data

In [35]:
fp_fit_data

epsilon      type sigma Fokker-Planck  \
                                  I_star   
0      1.0  gaussian   0.2      0.187525   
1      2.0  gaussian   0.2      0.212515   
2      4.0  gaussian   0.2      0.250000   
3      6.0  gaussian   0.2      0.250000   
4      8.0  gaussian   0.2      0.250000   
5     10.0  gaussian   0.2      0.250000   
6     12.0  gaussian   0.2      0.250000   
7     14.0  gaussian   0.2      0.250000   
8     16.0  gaussian   0.2      0.250000   
9     18.0  gaussian   0.2      0.250000   
10    20.0  gaussian   0.2      0.250000   
11    22.0  gaussian   0.2      0.250000   
12    24.0  gaussian   0.2      0.250000   
13    26.0  gaussian   0.2      0.250000   
14    28.0  gaussian   0.2      0.250000   
15    30.0  gaussian   0.2      0.250000   

                                                       
                                           time_scale  
0   [13.6, 13.59, 13.58, 13.57, 13.56, 13.55, 13.5...  
1   [20.900000000000002, 20.8, 20.7, 20.6, 20.5, 2...  
2   [6.79, 6.78, 6.7700000000000005, 6.76, 6.75, 6...  
3   [1.99, 1.98, 1.97, 1.96, 1.95, 1.94, 1.93, 1.9...  
4   [1.32, 1.31, 1.3, 1.29, 1.28, 1.27, 1.26, 1.25...  
5   [1.25, 1.24, 1.23, 1.22, 1.21, 1.2, 1.19, 1.18...  
6   [1.19, 1.18, 1.17, 1.16, 1.1500000000000001, 1...  
7   [1.22, 1.21, 1.2, 1.19, 1.18, 1.17, 1.16, 1.15...  
8   [1.16, 1.1500000000000001, 1.1400000000000001,...  
9   [1.19, 1.18, 1.17, 1.16, 1.1500000000000001, 1...  
10  [1.22, 1.21, 1.2, 1.19, 1.18, 1.17, 1.16, 1.15...  
11  [1.25, 1.24, 1.23, 1.22, 1.21, 1.2, 1.19, 1.18...  
12  [1.19, 1.18, 1.17, 1.16, 1.1500000000000001, 1...  
13  [1.1500000000000001, 1.1400000000000001, 1.130...  
14  [1.12, 1.11, 1.1, 1.09, 1.08, 1.07, 1.06, 1.05...  
15  [1.1, 1.09, 1.08, 1.07, 1.06, 1.05, 1.04, 1.03...

In [47]:
plt.figure()
plt.plot(fp_fit_data[("epsilon", "")], fp_fit_data[("Fokker-Planck", "I_star")])
plt.xlabel("$\\varepsilon$")
plt.ylabel("$I_\\ast$")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, '$I_\\ast$')

**Fact::the plot above shows that there are still A LOT of problems here!!!**

# FP process (moving $\kappa$)

### Setting dataframe

In [25]:
labels = (
    ("epsilon", ""),
    ("type", ""),
    ("sigma", ""),
    ("Fokker-Planck double fit", "k"),
    ("Fokker-Planck double fit", "I_star"),
    ("Fokker-Planck double fit", "time_scale")
)

fp_double_data = []
fp_double_fit_data = pd.DataFrame(columns=pd.MultiIndex.from_tuples(labels))

## Computing

In [26]:
selected_data = fitting_data.loc[fitting_data[("type", "")]=="gaussian"]

for i in tqdm(range(len(selected_data[("Model 4 (3 free pars)", "k")]))):
    k_const = selected_data[("Model 4 (3 free pars)", "k")].iloc[i]
    rho = selected_data[("Model 4 (3 free pars)", "rho")].iloc[i]
    N0 = selected_data[("Model 4 (3 free pars)", "N0")].iloc[i]
    
    I_max = cut_point**2 / 2
    I = np.linspace(0, I_max, 1000)
    I0 = I * np.exp(-(I/sigma**2))
    I0 /= integrate.trapz(I0, I)

    c = 1.0

    I_star_min = 0.01 ** 2
    I_star_max = 0.7 ** 2
    I_star_steps = 101
    I_star_list = np.linspace(I_star_min, I_star_max, I_star_steps)
    
    k_min = k_const - 0.02
    k_max = k_const + 0.1
    k_list = np.linspace(k_min, k_max, 20)

    iter_step = 10

    best_data = []
    best_time = []
    best_I_star = []
    best_error = []
    
    for q, k in tqdm(enumerate(k_list), total=len(k_list)):

        data = []
        errors = []
        original_times = []

        dt = 0.0001
        for j, I_star in tqdm(enumerate(I_star_list), total=I_star_steps):
            engine_nk = nk.cn_nekhoroshev(I_max, c, I_star, 1 / (k * 2), 0, I0, dt)
            step = 0

            t = []
            survival = []

            while(1.0 - engine_nk.get_particle_loss() >= lost_table_with_zero[i][0]):
                t.append(step * iter_step * dt)
                survival.append(1.0 - engine_nk.get_particle_loss())
                engine_nk.iterate(iter_step)
                #print("loss:", 1.0 - engine_nk.get_particle_loss())
                step += 1
                if step > 10000:
                    break
            t.append(step * iter_step * dt)
            survival.append(1.0 - engine_nk.get_particle_loss())

            print("Step:", j, "/", I_star_steps, ",", "I_star:{:.3}".format(I_star), ",", "Number of steps executed", step, ", dt:", dt)
            if step > 1000:
                print("Increasing dt!")
                dt *= 10

            f = interp1d(t, survival, kind="cubic")
            inv_f = inversefunc(
                f,
                domain=(t[0], t[-1]),
                image=(survival[-1], survival[0])
            )
            point_t = inv_f(lost_table[i][0])
            points_t = np.linspace(0, point_t, len(turn_sampling_with_zero))
            values_f = f(points_t)
            values_f = values_f[::-1]

            data.append(values_f)
            errors.append(chi_2(lost_table_with_zero[i], values_f))
            print("Error:", errors[-1])
            original_times.append(np.asarray(t)[::-1])
            if j > I_star_steps * 0.1:
                if(errors[-2] - errors[-1] < 0):
                    print("Found minimum! Breaking!")
                    break

        best_idx = np.argmin(errors)
        best_data.append(data[best_idx])
        best_time.append(original_times[best_idx])
        best_I_star.append(I_star_list[best_idx])
        best_error.append(errors[best_idx])
    
    absolute_best_idx = np.argmin(best_error)
    absolute_best_k = k_list[absolute_best_idx]
    absolute_best_data = best_data[absolute_best_idx]
    absolute_best_time = best_time[absolute_best_idx]
    absolute_best_I_star = best_I_star[absolute_best_idx]
    absolute_best_error = best_error[absolute_best_idx]
    
    fp_double_data.append(absolute_best_data)
    fp_double_fit_data.loc[len(fp_double_fit_data)] = [
        selected_data[("epsilon", "")].iloc[i],
        selected_data[("type", "")].iloc[i],
        selected_data[("sigma", "")].iloc[i],
        absolute_best_k,
        absolute_best_I_star,
        absolute_best_time
    ]
    

Step: 0 / 101 , I_star:0.0001 , Number of steps executed 83 , dt: 0.0001
Error: 1.9193850948752411
Step: 1 / 101 , I_star:0.005 , Number of steps executed 83 , dt: 0.0001
Error: 1.9497205999625316
Step: 2 / 101 , I_star:0.0099 , Number of steps executed 83 , dt: 0.0001
Error: 1.9514578079912386
Step: 3 / 101 , I_star:0.0148 , Number of steps executed 83 , dt: 0.0001
Error: 1.949642738458581
Step: 4 / 101 , I_star:0.0197 , Number of steps executed 83 , dt: 0.0001
Error: 1.94459210893877
Step: 5 / 101 , I_star:0.0246 , Number of steps executed 83 , dt: 0.0001
Error: 1.936542832494082
Step: 6 / 101 , I_star:0.0295 , Number of steps executed 84 , dt: 0.0001
Error: 1.9256947765141825
Step: 7 / 101 , I_star:0.0344 , Number of steps executed 85 , dt: 0.0001
Error: 1.912213552154211
Step: 8 / 101 , I_star:0.0393 , Number of steps executed 86 , dt: 0.0001
Error: 1.8962350841303388
Step: 9 / 101 , I_star:0.0442 , Number of steps executed 87 , dt: 0.0001
Error: 1.8778702277233095
Step: 10 / 101 ,

Step: 0 / 101 , I_star:0.0001 , Number of steps executed 83 , dt: 0.0001
Error: 1.9193840780116747
Step: 1 / 101 , I_star:0.005 , Number of steps executed 83 , dt: 0.0001
Error: 1.9497470990341468
Step: 2 / 101 , I_star:0.0099 , Number of steps executed 83 , dt: 0.0001
Error: 1.9514455634454984
Step: 3 / 101 , I_star:0.0148 , Number of steps executed 83 , dt: 0.0001
Error: 1.949773999896241
Step: 4 / 101 , I_star:0.0197 , Number of steps executed 83 , dt: 0.0001
Error: 1.944983168505635
Step: 5 / 101 , I_star:0.0246 , Number of steps executed 83 , dt: 0.0001
Error: 1.9372616504600946
Step: 6 / 101 , I_star:0.0295 , Number of steps executed 84 , dt: 0.0001
Error: 1.9267971461451372
Step: 7 / 101 , I_star:0.0344 , Number of steps executed 85 , dt: 0.0001
Error: 1.9137486894470623
Step: 8 / 101 , I_star:0.0393 , Number of steps executed 86 , dt: 0.0001
Error: 1.898248228615738
Step: 9 / 101 , I_star:0.0442 , Number of steps executed 87 , dt: 0.0001
Error: 1.8804050635150802
Step: 10 / 101

Step: 0 / 101 , I_star:0.0001 , Number of steps executed 83 , dt: 0.0001
Error: 1.9193829159826816
Step: 1 / 101 , I_star:0.005 , Number of steps executed 83 , dt: 0.0001
Error: 1.9497001397075666
Step: 2 / 101 , I_star:0.0099 , Number of steps executed 83 , dt: 0.0001
Error: 1.9513348723782726
Step: 3 / 101 , I_star:0.0148 , Number of steps executed 83 , dt: 0.0001
Error: 1.9498895939169154
Step: 4 / 101 , I_star:0.0197 , Number of steps executed 83 , dt: 0.0001
Error: 1.9453461814943633
Step: 5 / 101 , I_star:0.0246 , Number of steps executed 83 , dt: 0.0001
Error: 1.937932513809071
Step: 6 / 101 , I_star:0.0295 , Number of steps executed 84 , dt: 0.0001
Error: 1.9278319220479447
Step: 7 / 101 , I_star:0.0344 , Number of steps executed 85 , dt: 0.0001
Error: 1.9151945332072027
Step: 8 / 101 , I_star:0.0393 , Number of steps executed 86 , dt: 0.0001
Error: 1.9001476458339488
Step: 9 / 101 , I_star:0.0442 , Number of steps executed 87 , dt: 0.0001
Error: 1.8827987824833747
Step: 10 / 1

Step: 0 / 101 , I_star:0.0001 , Number of steps executed 83 , dt: 0.0001
Error: 1.9193816180924534
Step: 1 / 101 , I_star:0.005 , Number of steps executed 83 , dt: 0.0001
Error: 1.9495193327568232
Step: 2 / 101 , I_star:0.0099 , Number of steps executed 83 , dt: 0.0001
Error: 1.9512943409918946
Step: 3 / 101 , I_star:0.0148 , Number of steps executed 83 , dt: 0.0001
Error: 1.9499970255440775
Step: 4 / 101 , I_star:0.0197 , Number of steps executed 83 , dt: 0.0001
Error: 1.945675028954562
Step: 5 / 101 , I_star:0.0246 , Number of steps executed 83 , dt: 0.0001
Error: 1.9385568519374567
Step: 6 / 101 , I_star:0.0295 , Number of steps executed 84 , dt: 0.0001
Error: 1.9288022609689566
Step: 7 / 101 , I_star:0.0344 , Number of steps executed 85 , dt: 0.0001
Error: 1.9165553170415737
Step: 8 / 101 , I_star:0.0393 , Number of steps executed 86 , dt: 0.0001
Error: 1.9019387613763319
Step: 9 / 101 , I_star:0.0442 , Number of steps executed 87 , dt: 0.0001
Error: 1.88505820690419
Step: 10 / 101

Step: 0 / 101 , I_star:0.0001 , Number of steps executed 83 , dt: 0.0001
Error: 1.9193801958713634
Step: 1 / 101 , I_star:0.005 , Number of steps executed 83 , dt: 0.0001
Error: 1.9492210966734203
Step: 2 / 101 , I_star:0.0099 , Number of steps executed 83 , dt: 0.0001
Error: 1.9512934446562984
Step: 3 / 101 , I_star:0.0148 , Number of steps executed 83 , dt: 0.0001
Error: 1.9500721468666575
Step: 4 / 101 , I_star:0.0197 , Number of steps executed 83 , dt: 0.0001
Error: 1.9459775374008363
Step: 5 / 101 , I_star:0.0246 , Number of steps executed 83 , dt: 0.0001
Error: 1.939136738032227
Step: 6 / 101 , I_star:0.0295 , Number of steps executed 84 , dt: 0.0001
Error: 1.9297110517646572
Step: 7 / 101 , I_star:0.0344 , Number of steps executed 85 , dt: 0.0001
Error: 1.9178349125957572
Step: 8 / 101 , I_star:0.0393 , Number of steps executed 86 , dt: 0.0001
Error: 1.9036265532411414
Step: 9 / 101 , I_star:0.0442 , Number of steps executed 87 , dt: 0.0001
Error: 1.8871896323543216
Step: 10 / 1

...

Step: 0 / 101 , I_star:0.0001 , Number of steps executed 108 , dt: 0.0001
Error: 3.2642092424693496
Step: 1 / 101 , I_star:0.005 , Number of steps executed 114 , dt: 0.0001
Error: 3.29503552105122
Step: 2 / 101 , I_star:0.0099 , Number of steps executed 120 , dt: 0.0001
Error: 3.2965179966795244
Step: 3 / 101 , I_star:0.0148 , Number of steps executed 125 , dt: 0.0001
Error: 3.2956370344972608
Step: 4 / 101 , I_star:0.0197 , Number of steps executed 131 , dt: 0.0001
Error: 3.293940673531391
Step: 5 / 101 , I_star:0.0246 , Number of steps executed 136 , dt: 0.0001
Error: 3.2910504205888
Step: 6 / 101 , I_star:0.0295 , Number of steps executed 142 , dt: 0.0001
Error: 3.2868697682009884
Step: 7 / 101 , I_star:0.0344 , Number of steps executed 149 , dt: 0.0001
Error: 3.2813069085163646
Step: 8 / 101 , I_star:0.0393 , Number of steps executed 155 , dt: 0.0001
Error: 3.274340381077735
Step: 9 / 101 , I_star:0.0442 , Number of steps executed 162 , dt: 0.0001
Error: 3.2660041131381257
Step: 10

Step: 0 / 101 , I_star:0.0001 , Number of steps executed 108 , dt: 0.0001
Error: 3.2642100635717552
Step: 1 / 101 , I_star:0.005 , Number of steps executed 115 , dt: 0.0001
Error: 3.2949410630096887
Step: 2 / 101 , I_star:0.0099 , Number of steps executed 120 , dt: 0.0001
Error: 3.2963855872429124
Step: 3 / 101 , I_star:0.0148 , Number of steps executed 126 , dt: 0.0001
Error: 3.295453059697983
Step: 4 / 101 , I_star:0.0197 , Number of steps executed 131 , dt: 0.0001
Error: 3.293700396459086
Step: 5 / 101 , I_star:0.0246 , Number of steps executed 137 , dt: 0.0001
Error: 3.290804414499067
Step: 6 / 101 , I_star:0.0295 , Number of steps executed 143 , dt: 0.0001
Error: 3.2866609645118965
Step: 7 / 101 , I_star:0.0344 , Number of steps executed 150 , dt: 0.0001
Error: 3.281182557625232
Step: 8 / 101 , I_star:0.0393 , Number of steps executed 156 , dt: 0.0001
Error: 3.274345922727266
Step: 9 / 101 , I_star:0.0442 , Number of steps executed 163 , dt: 0.0001
Error: 3.266181681253568
Step: 1

Step: 0 / 101 , I_star:0.0001 , Number of steps executed 108 , dt: 0.0001
Error: 3.2642109836067834
Step: 1 / 101 , I_star:0.005 , Number of steps executed 115 , dt: 0.0001
Error: 3.294890028691117
Step: 2 / 101 , I_star:0.0099 , Number of steps executed 121 , dt: 0.0001
Error: 3.296225988262357
Step: 3 / 101 , I_star:0.0148 , Number of steps executed 126 , dt: 0.0001
Error: 3.2952785006757765
Step: 4 / 101 , I_star:0.0197 , Number of steps executed 132 , dt: 0.0001
Error: 3.2934596816667177
Step: 5 / 101 , I_star:0.0246 , Number of steps executed 138 , dt: 0.0001
Error: 3.2905581488727558
Step: 6 / 101 , I_star:0.0295 , Number of steps executed 144 , dt: 0.0001
Error: 3.2864496796468776
Step: 7 / 101 , I_star:0.0344 , Number of steps executed 151 , dt: 0.0001
Error: 3.281052863174197
Step: 8 / 101 , I_star:0.0393 , Number of steps executed 157 , dt: 0.0001
Error: 3.274342482718344
Step: 9 / 101 , I_star:0.0442 , Number of steps executed 164 , dt: 0.0001
Error: 3.2663459360263416
Step:

Step: 0 / 101 , I_star:0.0001 , Number of steps executed 108 , dt: 0.0001
Error: 3.264212004044004
Step: 1 / 101 , I_star:0.005 , Number of steps executed 115 , dt: 0.0001
Error: 3.2948763667373897
Step: 2 / 101 , I_star:0.0099 , Number of steps executed 121 , dt: 0.0001
Error: 3.296043979652706
Step: 3 / 101 , I_star:0.0148 , Number of steps executed 127 , dt: 0.0001
Error: 3.2951082285647004
Step: 4 / 101 , I_star:0.0197 , Number of steps executed 133 , dt: 0.0001
Error: 3.2932205339314704
Step: 5 / 101 , I_star:0.0246 , Number of steps executed 139 , dt: 0.0001
Error: 3.2903116143199735
Step: 6 / 101 , I_star:0.0295 , Number of steps executed 145 , dt: 0.0001
Error: 3.286236392334415
Step: 7 / 101 , I_star:0.0344 , Number of steps executed 151 , dt: 0.0001
Error: 3.280918308988279
Step: 8 / 101 , I_star:0.0393 , Number of steps executed 158 , dt: 0.0001
Error: 3.274330611031332
Step: 9 / 101 , I_star:0.0442 , Number of steps executed 165 , dt: 0.0001
Error: 3.266497482889638
Step: 1

Step: 0 / 101 , I_star:0.0001 , Number of steps executed 108 , dt: 0.0001
Error: 3.26421312621496
Step: 1 / 101 , I_star:0.005 , Number of steps executed 115 , dt: 0.0001
Error: 3.2948916767963583
Step: 2 / 101 , I_star:0.0099 , Number of steps executed 121 , dt: 0.0001
Error: 3.2958476768691107
Step: 3 / 101 , I_star:0.0148 , Number of steps executed 127 , dt: 0.0001
Error: 3.2949360149042373
Step: 4 / 101 , I_star:0.0197 , Number of steps executed 133 , dt: 0.0001
Error: 3.292984823246851
Step: 5 / 101 , I_star:0.0246 , Number of steps executed 139 , dt: 0.0001
Error: 3.2900649321388697
Step: 6 / 101 , I_star:0.0295 , Number of steps executed 146 , dt: 0.0001
Error: 3.28602151771475
Step: 7 / 101 , I_star:0.0344 , Number of steps executed 152 , dt: 0.0001
Error: 3.280779368130648
Step: 8 / 101 , I_star:0.0393 , Number of steps executed 159 , dt: 0.0001
Error: 3.274310836000938
Step: 9 / 101 , I_star:0.0442 , Number of steps executed 166 , dt: 0.0001
Error: 3.2666369075591133
Step: 10

## Visualize dataframe and data

In [34]:
fp_double_fit_data

epsilon      type sigma Fokker-Planck double fit            \
                                                  k    I_star   
0      1.0  gaussian   0.2                 0.143402  0.205858   
1      2.0  gaussian   0.2                 0.175470  0.230353   
2      4.0  gaussian   0.2                 0.220659  0.269545   
3      6.0  gaussian   0.2                 0.271919  0.333232   
4      8.0  gaussian   0.2                 0.329495  0.436111   
5     10.0  gaussian   0.2                 0.355003  0.490000   
6     12.0  gaussian   0.2                 0.367879  0.490000   
7     14.0  gaussian   0.2                 0.367879  0.490000   
8     16.0  gaussian   0.2                 0.387071  0.490000   
9     18.0  gaussian   0.2                 0.387071  0.490000   
10    20.0  gaussian   0.2                 0.387071  0.490000   
11    22.0  gaussian   0.2                 0.387071  0.490000   
12    24.0  gaussian   0.2                 0.406263  0.490000   
13    26.0  gaussian   0.2                 0.425455  0.490000   
14    28.0  gaussian   0.2                 0.444646  0.490000   
15    30.0  gaussian   0.2                 0.463838  0.490000   

                                                       
                                           time_scale  
0   [21.06, 21.05, 21.04, 21.03, 21.02, 21.01, 21....  
1   [39.1, 39.0, 38.9, 38.800000000000004, 38.7, 3...  
2   [77.60000000000001, 77.5, 77.4, 77.3, 77.2, 77...  
3   [239.0, 238.0, 237.0, 236.0, 235.0, 234.0, 233...  
4   [986.0, 985.0, 984.0, 983.0, 982.0, 981.0, 980...  
5   [1760.0, 1750.0, 1740.0, 1730.0, 1720.0, 1710....  
6   [819.0, 818.0, 817.0, 816.0, 815.0, 814.0, 813...  
7   [944.0, 943.0, 942.0, 941.0, 940.0, 939.0, 938...  
8   [330.0, 329.0, 328.0, 327.0, 326.0, 325.0, 324...  
9   [372.0, 371.0, 370.0, 369.0, 368.0, 367.0, 366...  
10  [414.0, 413.0, 412.0, 411.0, 410.0, 409.0, 408...  
11  [464.0, 463.0, 462.0, 461.0, 460.0, 459.0, 458...  
12  [190.0, 189.0, 188.0, 187.0, 186.0, 185.0, 184...  
13  [93.9, 93.8, 93.7, 93.60000000000001, 93.5, 93...  
14  [53.7, 53.6, 53.5, 53.4, 53.300000000000004, 5...  
15  [34.2, 34.1, 34.0, 33.9, 33.8, 33.7, 33.6, 33....

In [50]:
plt.figure()

plt.plot(
    selected_data[("epsilon", "")], selected_data[("Model 4 (3 free pars)", "k")],
    label="$\\kappa$ values from Model 4"
)

plt.plot(
    fp_double_fit_data[("epsilon", "")], fp_double_fit_data[("Fokker-Planck double fit", "k")],
    label="$\\kappa$ values from FP fitting process"
)

plt.ylabel("$\\kappa$")
plt.xlabel("$\\varepsilon$")
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [52]:
plt.figure()
plt.plot(fp_double_fit_data[("epsilon", "")], fp_double_fit_data[("Fokker-Planck double fit", "I_star")])
plt.xlabel("$\\varepsilon$")
plt.ylabel("$I_\\ast$")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, '$I_\\ast$')

**ALSO HERE WE OBSERVE THAT WE STILL NEED TO IMPROVE THE SCANNING PROCEDURE**